# DeepSeek OCR API Server for Google Colab

このノートブックはGoogle Colab上でDeepSeek OCRをAPIサーバーとして実行します。

## 使い方
1. ランタイム → ランタイムのタイプを変更 → GPU を選択
2. すべてのセルを順番に実行
3. ngrok URLをコピーして、MacBook ProのアプリケーションのDEEPSEEK_OCR_URLに設定

## 必要なもの
- Google アカウント
- ngrok アカウント（無料）: https://ngrok.com/


## 1. GPU確認

In [ ]:
!nvidia-smi

## 2. 依存関係のインストール

In [ ]:
# DeepSeek OCR依存関係
!pip install -q torch torchvision torchaudio
!pip install -q transformers==4.46.3
!pip install -q tokenizers==0.20.3
!pip install -q einops addict easydict
!pip install -q Pillow PyYAML opencv-python-headless
!pip install -q flask flask-cors pyngrok

# Flash Attention（オプション、高速化）
!pip install -q flash-attn==2.7.3 --no-build-isolation || echo "Flash Attention インストール失敗（スキップ）"

print("✅ インストール完了")

## 3. DeepSeek OCRプロセッサーの実装

In [ ]:
import os
import sys
import yaml
import re
from pathlib import Path
import torch
from transformers import AutoModel, AutoTokenizer
from PIL import Image
import io
import base64

class DeepSeekOCRProcessor:
    """DeepSeek-OCRを使用してLP画像を処理"""

    def __init__(self, model_name='deepseek-ai/DeepSeek-OCR'):
        print(f"Loading DeepSeek-OCR model: {model_name}")
        
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )

        self.model = AutoModel.from_pretrained(
            model_name,
            _attn_implementation='flash_attention_2',
            trust_remote_code=True,
            use_safetensors=True
        )

        self.model = self.model.eval().cuda().to(torch.bfloat16)
        print("✅ Model loaded successfully")

    def process_image_to_yaml(self, image_data, output_path='/tmp/deepseek_output'):
        """
        画像データ（base64またはバイト）からYAMLを生成
        """
        # 一時ファイルに保存
        os.makedirs(output_path, exist_ok=True)
        temp_image_path = os.path.join(output_path, 'temp_image.png')
        
        # 画像データを保存
        if isinstance(image_data, str):
            # base64デコード
            image_bytes = base64.b64decode(image_data)
            with open(temp_image_path, 'wb') as f:
                f.write(image_bytes)
        else:
            # バイトデータ
            with open(temp_image_path, 'wb') as f:
                f.write(image_data)
        
        # DeepSeek OCR実行
        prompt = "<image>\n<|grounding|>Convert the document to markdown. "
        
        result = self.model.infer(
            self.tokenizer,
            prompt=prompt,
            image_file=temp_image_path,
            output_path=output_path,
            base_size=1024,
            image_size=640,
            crop_mode=True,
            save_results=False,
            test_compress=True
        )
        
        # Markdownを取得
        if isinstance(result, dict):
            markdown_text = result.get('text', '') or result.get('content', '')
        else:
            markdown_text = str(result)
        
        # Markdown → YAML変換
        yaml_text = self.markdown_to_yaml(markdown_text)
        
        return {
            'yaml': yaml_text,
            'markdown': markdown_text
        }

    def markdown_to_yaml(self, markdown_text):
        """Markdown → YAML変換（簡易版）"""
        sections = []
        lines = markdown_text.split('\n')
        current_section = None
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            if line.startswith('#'):
                if current_section:
                    sections.append(current_section)
                
                heading_text = line.lstrip('#').strip()
                current_section = {
                    'type': 'content',
                    'texts': [{'content': heading_text, 'role': 'headline'}],
                    'buttons': [],
                    'items': []
                }
            elif current_section:
                current_section['texts'].append({'content': line, 'role': 'body'})
        
        if current_section:
            sections.append(current_section)
        
        yaml_data = {
            'meta': {'generator': 'DeepSeek-OCR'},
            'sections': {f'section{i+1}': s for i, s in enumerate(sections)}
        }
        
        return yaml.dump(yaml_data, allow_unicode=True, default_flow_style=False)

# グローバルインスタンス（初回ロード時のみ）
processor = None

def get_processor():
    global processor
    if processor is None:
        processor = DeepSeekOCRProcessor()
    return processor

print("✅ DeepSeekOCRProcessor定義完了")

## 4. Flask APIサーバーの実装

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import time

app = Flask(__name__)
CORS(app)  # CORS有効化

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'ok',
        'service': 'DeepSeek-OCR API Server',
        'gpu': torch.cuda.is_available(),
        'gpu_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else None
    })

@app.route('/ocr', methods=['POST'])
def ocr_endpoint():
    start_time = time.time()
    
    try:
        # ファイルアップロードまたはbase64データ
        if 'file' in request.files:
            file = request.files['file']
            image_data = file.read()
        elif 'image' in request.json:
            image_data = request.json['image']  # base64
        else:
            return jsonify({'success': False, 'error': 'No image provided'}), 400
        
        # DeepSeek OCR実行
        proc = get_processor()
        result = proc.process_image_to_yaml(image_data)
        
        processing_time = time.time() - start_time
        
        return jsonify({
            'success': True,
            'yaml': result['yaml'],
            'markdown': result['markdown'],
            'processingTime': processing_time * 1000,  # ms
            'metadata': {
                'modelType': 'DeepSeek-OCR',
                'processingTime': processing_time * 1000
            }
        })
    
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

print("✅ Flask APIサーバー定義完了")

## 5. ngrokのセットアップと起動

**重要:** ngrokの認証トークンを取得してください
1. https://ngrok.com/ でアカウント作成（無料）
2. ダッシュボードから認証トークンをコピー
3. 下のセルの `YOUR_NGROK_TOKEN` を置き換えて実行

In [ ]:
from pyngrok import ngrok, conf
import threading

# ngrok認証トークンを設定（https://dashboard.ngrok.com/get-started/your-authtokenから取得）
NGROK_AUTH_TOKEN = "YOUR_NGROK_TOKEN"  # ← ここを変更してください

if NGROK_AUTH_TOKEN == "YOUR_NGROK_TOKEN":
    print("⚠️ ngrok認証トークンを設定してください")
    print("1. https://ngrok.com/ でアカウント作成")
    print("2. ダッシュボードからトークンをコピー")
    print("3. 上のセルのNGROK_AUTH_TOKENを置き換えてください")
else:
    # ngrok設定
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    
    # モデルを事前ロード
    print("モデルをロード中...（初回は5-10分かかります）")
    get_processor()
    print("✅ モデルロード完了")
    
    # Flaskサーバーをバックグラウンドで起動
    def run_flask():
        app.run(port=5000)
    
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()
    
    # ngrokトンネル作成
    public_url = ngrok.connect(5000)
    
    print("\n" + "="*60)
    print("🎉 DeepSeek OCR APIサーバー起動完了！")
    print("="*60)
    print(f"\n📡 公開URL: {public_url}")
    print(f"\nヘルスチェック: {public_url}/health")
    print(f"OCRエンドポイント: {public_url}/ocr")
    print("\n⚠️ このセルを実行し続けている限りサーバーは稼働します")
    print("⚠️ Colabセッションが切れたら再起動が必要です")
    print("\n💡 MacBook Proのアプリケーションで以下を設定:")
    print(f"   DEEPSEEK_OCR_URL={public_url}")
    print("="*60 + "\n")
    
    # サーバーを稼働し続ける
    import time
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nサーバーを停止しました")

## テスト（オプション）

APIが正常に動作しているかテストします。

In [ ]:
import requests

# 上で表示された公開URLを使用
API_URL = "YOUR_NGROK_URL"  # 例: https://xxxx-xx-xx-xx-xx.ngrok.io

# ヘルスチェック
response = requests.get(f"{API_URL}/health")
print("ヘルスチェック結果:")
print(response.json())